In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.model_selection import cross_val_score,cross_val_predict, train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer,LabelEncoder

from sklearn.metrics import accuracy_score,classification_report, recall_score,confusion_matrix, roc_auc_score, precision_score, f1_score, roc_curve, auc, plot_confusion_matrix,plot_roc_curve


import optuna
import lightgbm as lgb
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier, plot_importance
from catboost import CatBoostClassifier




#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import shap 

import missingno as msno

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")

In [4]:
df = data.copy()

In [5]:
df.head()

In [8]:
df.info()

In [ ]:
df['Age'] = df['Age'].astype(float)
df['MaxHR'] = df['MaxHR'].astype(float)
df['Cholesterol'] = df['Cholesterol'].astype(float)
df['RestingBP'] = df['RestingBP'].astype(float)
df['FastingBS'] = df['FastingBS'].astype(float)



In [ ]:
df1 = df.copy()

In [9]:
accuracy= []
recall =[]
roc_auc= []
precision = []


df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df1 = df.copy()
le = LabelEncoder()
df1['HeartDisease']=le.fit_transform(df1['HeartDisease'])

df['Age'] = df['Age'].astype(float)
df['MaxHR'] = df['MaxHR'].astype(float)
df['Cholesterol'] = df['Cholesterol'].astype(float)
df['RestingBP'] = df['RestingBP'].astype(float)
df['FastingBS'] = df['FastingBS'].astype(float)





X= df1.drop('HeartDisease', axis=1)
y= df1['HeartDisease']

categorical_features_indices = np.where(X.dtypes != np.float)[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

catboost_5 = CatBoostClassifier(verbose=False,random_state=0)

catboost_5.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_test, y_test),early_stopping_rounds=100)
y_pred = catboost_5.predict(X_test)

accuracy.append(round(accuracy_score(y_test, y_pred),4))
recall.append(round(recall_score(y_test, y_pred),4))
roc_auc.append(round(roc_auc_score(y_test, y_pred),4))
precision.append(round(precision_score(y_test, y_pred),4))

model_names = ['Catboost_adjusted_weight_5']
result_df3 = pd.DataFrame({'Accuracy':accuracy,'Recall':recall, 'Roc_Auc':roc_auc, 'Precision':precision}, index=model_names)
result_df3

In [11]:
def objective(trial):
    
    df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
    df1 = df.copy()
    le = LabelEncoder()
    df1['HeartDisease']=le.fit_transform(df1['HeartDisease'])

    df['Age'] = df['Age'].astype(float)
    df['MaxHR'] = df['MaxHR'].astype(float)
    df['Cholesterol'] = df['Cholesterol'].astype(float)
    df['RestingBP'] = df['RestingBP'].astype(float)
    df['FastingBS'] = df['FastingBS'].astype(float)


    X= df1.drop('HeartDisease', axis=1)
    y= df1['HeartDisease']

    categorical_features_indices = np.where(X.dtypes != np.float)[0]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    param = {
        "objective": "Logloss",
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat_cls = CatBoostClassifier(verbose=False,random_state=0,**param)

    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)], cat_features=categorical_features_indices,verbose=0, early_stopping_rounds=100)

    preds = cat_cls.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [12]:
accuracy= []
recall =[]
roc_auc= []
precision = []


df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df1 = df.copy()
le = LabelEncoder()
df1['HeartDisease']=le.fit_transform(df1['HeartDisease'])

df['Age'] = df['Age'].astype(float)
df['MaxHR'] = df['MaxHR'].astype(float)
df['Cholesterol'] = df['Cholesterol'].astype(float)
df['RestingBP'] = df['RestingBP'].astype(float)
df['FastingBS'] = df['FastingBS'].astype(float)





X= df1.drop('HeartDisease', axis=1)
y= df1['HeartDisease']

categorical_features_indices = np.where(X.dtypes != np.float)[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

catboost_5 = CatBoostClassifier(verbose=False,random_state=0,colsample_bylevel= 0.059530780006608025,
    depth = 10,
    boosting_type = "Plain",
    bootstrap_type= "MVS")

catboost_5.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_test, y_test),early_stopping_rounds=100)
y_pred = catboost_5.predict(X_test)

accuracy.append(round(accuracy_score(y_test, y_pred),4))
recall.append(round(recall_score(y_test, y_pred),4))
roc_auc.append(round(roc_auc_score(y_test, y_pred),4))
precision.append(round(precision_score(y_test, y_pred),4))

model_names = ['Catboost_adjusted_weight_5']
result_df3 = pd.DataFrame({'Accuracy':accuracy,'Recall':recall, 'Roc_Auc':roc_auc, 'Precision':precision}, index=model_names)
result_df3

In [ ]:
accuracy= []
recall =[]
roc_auc= []
precision = []


df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df1 = df.copy()
le = LabelEncoder()
df1['HeartDisease']=le.fit_transform(df1['HeartDisease'])

df['Age'] = df['Age'].astype(float)
df['MaxHR'] = df['MaxHR'].astype(float)
df['Cholesterol'] = df['Cholesterol'].astype(float)
df['RestingBP'] = df['RestingBP'].astype(float)
df['FastingBS'] = df['FastingBS'].astype(float)





X= df1.drop('HeartDisease', axis=1)
y= df1['HeartDisease']


for col in X.columns:
    col_type = X[col].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lgbmc_5=lgb.LGBMClassifier(random_state=0,scale_pos_weight=5, num_leaves= 2)

lgbmc_5.fit(X_train, y_train,categorical_feature = 'auto',eval_set=(X_test, y_test),feature_name='auto', verbose=0)

y_pred = lgbmc_5.predict(X_test)

accuracy.append(round(accuracy_score(y_test, y_pred),4))
recall.append(round(recall_score(y_test, y_pred),4))
roc_auc.append(round(roc_auc_score(y_test, y_pred),4))
precision.append(round(precision_score(y_test, y_pred),4))

model_names = ['LightGBM_adjusted_weight_5']
result_df9 = pd.DataFrame({'Accuracy':accuracy,'Recall':recall, 'Roc_Auc':roc_auc, 'Precision':precision}, index=model_names)
result_df9

In [17]:
X.info()

In [15]:
for col in X.columns:
    col_type = X[col].dtype
    print(col_type.name)
    #if col_type == 'object' or col_type.name == 'category':
       # X[col] = X[col].astype('category')



In [ ]:
def objective(trial):
    df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
    df1 = df.copy()
    le = LabelEncoder()
    df1['HeartDisease']=le.fit_transform(df1['HeartDisease'])

    df['Age'] = df['Age'].astype(float)
    df['MaxHR'] = df['MaxHR'].astype(float)
    df['Cholesterol'] = df['Cholesterol'].astype(float)
    df['RestingBP'] = df['RestingBP'].astype(float)
    df['FastingBS'] = df['FastingBS'].astype(float)
    
    for col in X.columns:
        col_type = X[col].dtype
        if col_type == 'object' or col_type.name == 'category':
            X[col] = X[col].astype('category')    
    
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "dart",
        "num_leaves": trial.suggest_int("num_leaves", 2,2000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    lgbmc_adj=lgb.LGBMClassifier(random_state=0,**param)
    lgbmc_adj.fit(X_train, y_train,categorical_feature = 'auto',eval_set=(X_test, y_test),feature_name='auto', verbose=0)

    preds = lgbmc_adj.predict(X_test)
    pred_labels = np.rint(preds)
    recall = recall_score(y_test, pred_labels)
    return recall


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
accuracy= []
recall =[]
roc_auc= []
precision = []


df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df1 = df.copy()
le = LabelEncoder()
df1['HeartDisease']=le.fit_transform(df1['HeartDisease'])

df['Age'] = df['Age'].astype(float)
df['MaxHR'] = df['MaxHR'].astype(float)
df['Cholesterol'] = df['Cholesterol'].astype(float)
df['RestingBP'] = df['RestingBP'].astype(float)
df['FastingBS'] = df['FastingBS'].astype(float)





X= df1.drop('HeartDisease', axis=1)
y= df1['HeartDisease']


for col in X.columns:
    col_type = X[col].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lgbmc_5=lgb.LGBMClassifier(random_state=0, num_leaves= 495,
    max_depth= 10,
    lambda_l1= 1.462539905997426,
    lambda_l2= 2.5068794797011424,
    feature_fraction= 0.512368850608734,
    bagging_fraction= 0.701320405856735,
    bagging_freq= 3,
    min_child_samples= 11)

lgbmc_5.fit(X_train, y_train,categorical_feature = 'auto',eval_set=(X_test, y_test),feature_name='auto', verbose=0,early_stopping_rounds=)

y_pred = lgbmc_5.predict(X_test)

accuracy.append(round(accuracy_score(y_test, y_pred),4))
recall.append(round(recall_score(y_test, y_pred),4))
roc_auc.append(round(roc_auc_score(y_test, y_pred),4))
precision.append(round(precision_score(y_test, y_pred),4))

model_names = ['LightGBM_adjusted_weight_5']
result_df9 = pd.DataFrame({'Accuracy':accuracy,'Recall':recall, 'Roc_Auc':roc_auc, 'Precision':precision}, index=model_names)
result_df9